In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [4]:
# build the dataset
block_size = 3    # context length: number of characters we take to predict the next one.

def build_dataset(words):
    X, Y = [], []
    
    for w in words:
    
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]   # crop and append
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [5]:
Xtr.shape, Ytr.shape  # Dataset

(torch.Size([182625, 3]), torch.Size([182625]))

In [6]:
g = torch.Generator().manual_seed(2147483647)   # for reproducibility
C = torch.randn((27,10), generator = g)

W1 = torch.randn((30, 200), generator = g)
b1 = torch.randn(200, generator = g)
W2 = torch.randn((200, 27), generator = g)
b2 = torch.randn(27, generator = g)

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [7]:
sum(p.nelement() for p in parameters)   # number of parameters in total

11897

In [8]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [9]:
for i in range(200000):
    #minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # Forward Pass
    emb = C[Xtr[ix]]    # (32, 3 ,2)
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)  # (32, 100)
    logits = h @ W2 + b2    # (32, 27)
    loss = F.cross_entropy(logits, Ytr[ix])
    
    # Backward Pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # Update the parameters
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

In [10]:
# Loss of training dataset after training
emb = C[Xtr]   
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)  # (32, 100)
logits = h @ W2 + b2    # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.1300, grad_fn=<NllLossBackward0>)

In [11]:
# Loss of dev dataset after training
emb = C[Xdev]   
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)  # (32, 100)
logits = h @ W2 + b2    # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1687, grad_fn=<NllLossBackward0>)

In [12]:
# Sample from the model

g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(10):

    out = []
    context = [0] * block_size # Initialize with all ...
    while True:
        emb = C[torch.tensor([context])]  # (1, block_size, d)
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim = 1)
        ix = torch.multinomial(probs, num_samples = 1, generator = g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

mohaag.
yazhiee.
med.
ryah.
rethan.
ejerlee.
azeerdielin.
shi.
jen.
edelissopharleigh.
